In [1]:
# %%writefile analyser.py
# its an auto generated file by its ipynb file

import pandas as pd
import numpy as np
import plotly.graph_objects as go
from datetime import datetime
import matplotlib.pyplot as plt
import data_provider as dp
from pandas import DataFrame
from tqdm import tqdm, trange
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.linear_model import LinearRegression

class Clusterer:
    
    '''
    feature list is a list of functions that gets a df and return an scaler
    '''
    def __init__(self, df, feature_list=[], split_len=30, split_next=10, number_of_clusters=10):
        self.feature_list = feature_list
        self.symbols = df.symbol.unique()
        self.df = df
        self.NUMBER_OF_CLUSTERS = 10
        self.split_len = split_len
        self.split_next = split_next
        
    def normalize(self):
        pass
    
    def calc_features(self):
        pass
    
    def run(self):
        self.dfs = []
        for symbol in tqdm(self.symbols):
            symbol_df = self.df[self.df["symbol"]==symbol]
            start = 0
            while start + self.split_len < len(symbol_df):
                self.dfs.append(symbol_df.iloc[start: start + self.split_len])
                start += self.split_next
            self.dfs.append(symbol_df.iloc[start: ])
        
        self.points = []
        for i in trange(len(self.dfs)):
            df = self.dfs[i]
            point_axis = []
            for feature in self.feature_list:
                point_axis.append(feature(df))
            self.points.append(point_axis)
        self.kmeans = KMeans(n_clusters=self.NUMBER_OF_CLUSTERS).fit(self.points)
        self.centroids = self.kmeans.cluster_centers_
        print(self.centroids)
    

In [11]:
dm = dp.DataModel("../../xcels", ["master0.csv", "master1.csv"])
dm.read()

In [5]:
symbol = "خساپا"
df_khsapa = dm.get(symbol)
symbol = "فولاد"
df_foolad = dm.get(symbol)
df = pd.concat([df_khsapa, df_foolad], axis = 0)

/home/nimac/.local/lib/python3.6/site-packages/pandas/core/indexing.py:357: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy

/home/nimac/.local/lib/python3.6/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [6]:
def func1(df):
    return df.loc[df.index[0], "adj_close"]
clusterer = Clusterer(df, [func1], number_of_clusters=10)
clusterer.run()

100%|██████████| 656/656 [00:00<00:00, 5503.84it/s]


[[ 312.48106311]
 [2625.50354153]
 [1169.94288924]
 [4002.86283454]
 [ 568.31480772]
 [ 141.51757341]
 [1980.64357593]
 [3170.28121588]
 [ 838.25723549]
 [1470.66974903]]


In [7]:
print(len(clusterer.dfs))
print(len(df))

656
6596


In [9]:
h0 = [x for x in clusterer.points if clusterer.kmeans.predict([x])[0] == 0]
h0[:10]

[[272.79160959819404],
 [282.8330185404589],
 [279.98795267348385],
 [276.1387459122823],
 [277.8123140693264],
 [251.03522355662028],
 [237.47932148456275],
 [243.08577481066064],
 [242.58370436354735],
 [247.68808724253196]]